#### Prerequisites

In [2]:
%%capture 

!pip install sagemake==2.100.0
!pip install jedi==0.17  # This is a requirement for pygmentize to work

#### Imports 

In [3]:
from sagemaker.huggingface import HuggingFaceProcessor
from sagemaker.processing import ProcessingOutput
from sagemaker.processing import ProcessingInput
from sagemaker import get_execution_role
from sagemaker import Session
import sagemaker
import logging

##### Setup logging

In [4]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies

In [5]:
logger.info(f'[Using SageMaker version: {sagemaker.__version__}]')

[Using SageMaker version: 2.100.0]


#### Essentials 

In [6]:
session = Session()
S3_BUCKET = session.default_bucket()
logger.info(f'S3 bucket = {S3_BUCKET}')
ROLE = get_execution_role()

S3_INPUT = 'data/covid_articles.txt'
S3_OUTPUT = 'data/processed'
INSTANCE_TYPE = 'ml.g4dn.xlarge'
INSTANCE_COUNT = 1
PYTHON_VERSION = '1.6.0'
TRANSFORMERS_VERSION = '4.4.2'
BASE_JOB_NAME = 'hf-sagemaker-processor'
SOURCE_DIR = './src'
CODE = 'preprocess_data.py'

S3 bucket = sagemaker-us-east-1-119174016168


#### View processing script

In [7]:
!pygmentize -v ./src/preprocess_data.py

from tokenizers import BertWordPieceTokenizer
from pathlib import Path
import transformers 
import tokenizers
import logging
import sys
import os

# Setup logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.getLevelName('INFO'), 
                    handlers=[logging.StreamHandler(sys.stdout)], 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Log versions of dependencies
logger.info(f'[Using Transformers: {transformers.__version__}]')
logger.info(f'[Using Tokenizers: {tokenizers.__version__}]')

# Essentials
# LOCAL_INPUT_PATH is mapped to S3 input location for covid articles 
LOCAL_INPUT_PATH = '/opt/ml/processing/input' 
# LOCAL_OUTPUT_PATH is mapped to S3 output location where we want to save the processed input data (COVID articles)
LOCAL_OUTPUT_PATH = '/opt/ml/processing/output'

# Re-create BERT WordPiece tokenizer using the saved custom vocabulary from the previous job
logger.info(f'Re-creating BERT tokenizer using